Importing the required Modules

In [56]:
import random
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import SGD
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd


In [57]:
lemmatizer = WordNetLemmatizer()

Download the nltk packages

In [58]:
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Init File of Intents.json

In [59]:
words = []
classes = []
documents = []
ignore_words = ["?", "!",","]

read the intents.json file

In [60]:
data_file = open("../intents.json").read()
data_file

'{\n  "intent": [\n    {\n      "tag": "greeting",\n      "patterns": [\n        "Hi",\n        "How are you?",\n        "Hey there, how have you been?",\n        "Is anyone there?",\n        "Hello",\n        "Good day",\n        "Hiii",\n        "Hii"\n      ],\n      "responses": [\n        "Hi there, what can I do for you?",\n        "Hey there, how can I help?"\n      ]\n    },\n    {\n      "tag": "goodbye",\n      "patterns": ["Bye", "See you there", "Goodbye"],\n      "responses": [\n        "See you later, thanks for visiting ",\n        "Have a nice day",\n        "Bye! Come back again soon."\n      ]\n    },\n    {\n      "tag": "thanks",\n      "patterns": ["Thanks", "Thank you", "That\'s helpful", "Nice", "Good"],\n      "responses": ["Happy to help!", "Any time!", "My pleasure"]\n    },\n    {\n      "tag": "about",\n      "patterns": [\n        "about you",\n        "Who are you?",\n        "What are you?",\n        "Who you are?",\n        "what you can do",\n        "W

Converting the intents file from bytes to json 

In [61]:
intents=json.loads(data_file)
intents

{'intent': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Hey there, how have you been?',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Hiii',
    'Hii'],
   'responses': ['Hi there, what can I do for you?',
    'Hey there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you there', 'Goodbye'],
   'responses': ['See you later, thanks for visiting ',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", 'Nice', 'Good'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'about',
   'patterns': ['about you',
    'Who are you?',
    'What are you?',
    'Who you are?',
    'what you can do',
    'What is the Purpose of the project'],
   'responses': ['I am AyurBot. I can help determine your prakriti type. Prakriti is our inherent nature, with which we are born. It is the nature of our body in terms of dosha. It dies not change duri

In [62]:
for intent in intents["intent"]:
    for pattern in intent["patterns"]:
        # separate each word from the pattern
        w=nltk.word_tokenize(pattern)
        # add it in the words list
        words.extend(w)
        
        # adding it in the documents with the defined tag.
        documents.append((w,intent["tag"]))

        # adding the classes in the classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [63]:
words

['Hi',
 'How',
 'are',
 'you',
 '?',
 'Hey',
 'there',
 ',',
 'how',
 'have',
 'you',
 'been',
 '?',
 'Is',
 'anyone',
 'there',
 '?',
 'Hello',
 'Good',
 'day',
 'Hiii',
 'Hii',
 'Bye',
 'See',
 'you',
 'there',
 'Goodbye',
 'Thanks',
 'Thank',
 'you',
 'That',
 "'s",
 'helpful',
 'Nice',
 'Good',
 'about',
 'you',
 'Who',
 'are',
 'you',
 '?',
 'What',
 'are',
 'you',
 '?',
 'Who',
 'you',
 'are',
 '?',
 'what',
 'you',
 'can',
 'do',
 'What',
 'is',
 'the',
 'Purpose',
 'of',
 'the',
 'project',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'I',
 'call',
 'you',
 'whats',
 'your',
 'name',
 '?',
 'Could',
 'you',
 'help',
 'me',
 '?',
 'give',
 'me',
 'a',
 'hand',
 'please',
 'Can',
 'you',
 'help',
 '?',
 'What',
 'can',
 'you',
 'do',
 'for',
 'me',
 '?',
 'I',
 'need',
 'a',
 'support',
 'I',
 'need',
 'a',
 'help',
 'support',
 'me',
 'please',
 'Complete',
 'the',
 'work',
 'My',
 'name',
 'user',
 'name',
 'you',
 'are',
 'not',
 'Good',
 'Nothing',
 'You',
 'are',
 

In [64]:
for doc in documents:
    print(doc)

(['Hi'], 'greeting')
(['How', 'are', 'you', '?'], 'greeting')
(['Hey', 'there', ',', 'how', 'have', 'you', 'been', '?'], 'greeting')
(['Is', 'anyone', 'there', '?'], 'greeting')
(['Hello'], 'greeting')
(['Good', 'day'], 'greeting')
(['Hiii'], 'greeting')
(['Hii'], 'greeting')
(['Bye'], 'goodbye')
(['See', 'you', 'there'], 'goodbye')
(['Goodbye'], 'goodbye')
(['Thanks'], 'thanks')
(['Thank', 'you'], 'thanks')
(['That', "'s", 'helpful'], 'thanks')
(['Nice'], 'thanks')
(['Good'], 'thanks')
(['about', 'you'], 'about')
(['Who', 'are', 'you', '?'], 'about')
(['What', 'are', 'you', '?'], 'about')
(['Who', 'you', 'are', '?'], 'about')
(['what', 'you', 'can', 'do'], 'about')
(['What', 'is', 'the', 'Purpose', 'of', 'the', 'project'], 'about')
(['what', 'is', 'your', 'name'], 'name')
(['what', 'should', 'I', 'call', 'you'], 'name')
(['whats', 'your', 'name', '?'], 'name')
(['Could', 'you', 'help', 'me', '?'], 'help')
(['give', 'me', 'a', 'hand', 'please'], 'help')
(['Can', 'you', 'help', '?'], 'h

In [65]:
classes

['greeting',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'noanswer',
 'prakriti',
 'creators',
 'badwords',
 'acknowledge',
 'age_inquiry',
 'positive_feedback',
 'negative_feedback',
 'information_request',
 'contact_support',
 'diet_recommendation',
 'exercise_suggestions',
 'stress_management',
 'sleep_tips',
 'nutrition_information',
 'exit']

Lemmatization on words and removing the words which are not usable

In [66]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words

['hi',
 'how',
 'are',
 'you',
 'hey',
 'there',
 'how',
 'have',
 'you',
 'been',
 'is',
 'anyone',
 'there',
 'hello',
 'good',
 'day',
 'hiii',
 'hii',
 'bye',
 'see',
 'you',
 'there',
 'goodbye',
 'thanks',
 'thank',
 'you',
 'that',
 "'s",
 'helpful',
 'nice',
 'good',
 'about',
 'you',
 'who',
 'are',
 'you',
 'what',
 'are',
 'you',
 'who',
 'you',
 'are',
 'what',
 'you',
 'can',
 'do',
 'what',
 'is',
 'the',
 'purpose',
 'of',
 'the',
 'project',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'i',
 'call',
 'you',
 'whats',
 'your',
 'name',
 'could',
 'you',
 'help',
 'me',
 'give',
 'me',
 'a',
 'hand',
 'please',
 'can',
 'you',
 'help',
 'what',
 'can',
 'you',
 'do',
 'for',
 'me',
 'i',
 'need',
 'a',
 'support',
 'i',
 'need',
 'a',
 'help',
 'support',
 'me',
 'please',
 'complete',
 'the',
 'work',
 'my',
 'name',
 'user',
 'name',
 'you',
 'are',
 'not',
 'good',
 'nothing',
 'you',
 'are',
 'bad',
 'noting',
 'to',
 'worry',
 'chatgpt',
 'is',
 'best',
 'i

Sorting the words

In [67]:
words = sorted(list(set(words)))
words

["'m",
 "'re",
 "'s",
 'a',
 'about',
 'age',
 'annoying',
 'anyone',
 'are',
 'away',
 'bad',
 'be',
 'been',
 'best',
 'by',
 'bye',
 'ca',
 'call',
 'can',
 'chatbot',
 'chatgpt',
 'complete',
 'contact',
 'could',
 'created',
 'creator',
 'customer',
 'day',
 'despise',
 'detail',
 'determine',
 'diet',
 'disappointed',
 'do',
 'doing',
 'eat',
 'eating',
 'exercise',
 'exit',
 'fact',
 'feeling',
 'figure',
 'find',
 'food',
 'for',
 'fuck',
 'get',
 'give',
 'go',
 'good',
 'goodbye',
 'great',
 'hand',
 'hate',
 'have',
 'healthy',
 'hello',
 'help',
 'helpful',
 'hey',
 'hi',
 'hii',
 'hiii',
 'how',
 'i',
 'identify',
 'impressed',
 'improve',
 'in',
 'information',
 'irrelevant',
 'is',
 'job',
 'learn',
 'love',
 'made',
 'make',
 'manage',
 'me',
 'more',
 'my',
 "n't",
 'name',
 'need',
 'nice',
 'not',
 'nothing',
 'noting',
 'now',
 'nutrition',
 'of',
 'ok',
 'old',
 'out',
 'over',
 'plan',
 'please',
 'prakriti',
 'project',
 'provide',
 'purpose',
 'quit',
 'reach',


Sorting the Classes

In [68]:
classes = sorted(list(set(classes)))
classes

['about',
 'acknowledge',
 'age_inquiry',
 'badwords',
 'contact_support',
 'creators',
 'diet_recommendation',
 'exercise_suggestions',
 'exit',
 'goodbye',
 'greeting',
 'help',
 'information_request',
 'name',
 'negative_feedback',
 'noanswer',
 'nutrition_information',
 'positive_feedback',
 'prakriti',
 'sleep_tips',
 'stress_management',
 'thanks']

Classes and words and Documents overview

In [69]:
print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)

101 documents
22 classes ['about', 'acknowledge', 'age_inquiry', 'badwords', 'contact_support', 'creators', 'diet_recommendation', 'exercise_suggestions', 'exit', 'goodbye', 'greeting', 'help', 'information_request', 'name', 'negative_feedback', 'noanswer', 'nutrition_information', 'positive_feedback', 'prakriti', 'sleep_tips', 'stress_management', 'thanks']
148 unique lemmatized words ["'m", "'re", "'s", 'a', 'about', 'age', 'annoying', 'anyone', 'are', 'away', 'bad', 'be', 'been', 'best', 'by', 'bye', 'ca', 'call', 'can', 'chatbot', 'chatgpt', 'complete', 'contact', 'could', 'created', 'creator', 'customer', 'day', 'despise', 'detail', 'determine', 'diet', 'disappointed', 'do', 'doing', 'eat', 'eating', 'exercise', 'exit', 'fact', 'feeling', 'figure', 'find', 'food', 'for', 'fuck', 'get', 'give', 'go', 'good', 'goodbye', 'great', 'hand', 'hate', 'have', 'healthy', 'hello', 'help', 'helpful', 'hey', 'hi', 'hii', 'hiii', 'how', 'i', 'identify', 'impressed', 'improve', 'in', 'informatio

In [70]:
documents

[(['Hi'], 'greeting'),
 (['How', 'are', 'you', '?'], 'greeting'),
 (['Hey', 'there', ',', 'how', 'have', 'you', 'been', '?'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Hiii'], 'greeting'),
 (['Hii'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'there'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Nice'], 'thanks'),
 (['Good'], 'thanks'),
 (['about', 'you'], 'about'),
 (['Who', 'are', 'you', '?'], 'about'),
 (['What', 'are', 'you', '?'], 'about'),
 (['Who', 'you', 'are', '?'], 'about'),
 (['what', 'you', 'can', 'do'], 'about'),
 (['What', 'is', 'the', 'Purpose', 'of', 'the', 'project'], 'about'),
 (['what', 'is', 'your', 'name'], 'name'),
 (['what', 'should', 'I', 'call', 'you'], 'name'),
 (['whats', 'your', 'name', '?'], 'name'),
 (['Could', 'you', 'help', 'me', '?'], 'help'),
 (['give', 'me', 'a', 'hand'

dumping the words list and classes list

In [71]:
# pickle.dump(words, open("words.pkl", "wb"))
# pickle.dump(classes, open("classes.pkl", "wb"))

Training initializer

In [72]:
training=[]

In [73]:
output_empty=[0]*len(classes)
output_empty

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [74]:
for doc in documents:
    bag=[]
    # extracting the patterns
    pattern_words=doc[0]
    
    # lemmatizing the words from the pattern
    pattern_words=[lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
     # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])
    
print(training)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

Shuffling our features and turn into np.array

In [75]:
# random.shuffle(training)
print(training[0][0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [76]:
print(training[0])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [77]:
for train in training:
    print(train)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0

In [78]:
training_data=np.array(training,dtype=object)
training_data

array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [79]:
training_data.shape

(101, 2)

Training X and Training Y data

In [80]:
train_x=list(training_data[:,0])
train_x

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [81]:
train_y=list(training_data[:,1])
train_y

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Creating the Model

In [82]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

Summary of the model

In [83]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               19072     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 22)                1430      
                                                                 
Total params: 28758 (112.34 KB)
Trainable params: 28758 (112.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

In [84]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [85]:
model.fit(np.array(train_x),np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200


21/21 [==============================] - 1s 2ms/step - loss: 3.0796 - accuracy: 0.0594 
Epoch 2/200
21/21 [==============================] - 0s 5ms/step - loss: 3.0873 - accuracy: 0.0594
Epoch 3/200
21/21 [==============================] - 0s 2ms/step - loss: 2.9841 - accuracy: 0.0990
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 2.9469 - accuracy: 0.1782
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 2.8477 - accuracy: 0.1485
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 2.8246 - accuracy: 0.1287
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 2.6575 - accuracy: 0.2871
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 2.6479 - accuracy: 0.1980
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 2.5570 - accuracy: 0.2178
Epoch 10/200
21/21 [==============================] - 0s 3ms/step - loss: 2.4903 - accuracy: 0.2277
Epoch 11/200
21/21 [

In [86]:
np.array(train_x)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

The Model is Trained

Lets predict it using the model

In [87]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [88]:
clean_up_sentence("hello who are you")

['hello', 'who', 'are', 'you']

In [89]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


In [90]:
bow("hello who are you",words)

found in bag: hello
found in bag: who
found in bag: are
found in bag: you


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0])

In [91]:
words[8]

'are'

In [92]:
def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    print(res)
    index=np.argmax(res)
    probmax=np.max(res)
    print("maximum index =",index,"probabiliy = ",probmax)
    results={"intent": classes[index], "probability": str(probmax)}
    return results


In [93]:
predict_class("stupid bot",model)

1/1 [==============================] - 0s 77ms/step
[1.6769778e-03 6.9250003e-02 5.6960792e-03 1.4154404e-02 2.0683878e-03
 4.5999959e-03 1.3237487e-03 2.1827954e-04 8.0603957e-02 4.7268443e-02
 6.2666863e-01 1.2104318e-04 1.6269991e-04 6.8012514e-04 9.8488666e-04
 3.5844527e-02 3.3920960e-04 4.3283179e-03 1.6242097e-03 4.9097469e-04
 3.8144349e-03 9.8080620e-02]
maximum index = 10 probabiliy =  0.62666863


{'intent': 'greeting', 'probability': '0.62666863'}

In [94]:
classes

['about',
 'acknowledge',
 'age_inquiry',
 'badwords',
 'contact_support',
 'creators',
 'diet_recommendation',
 'exercise_suggestions',
 'exit',
 'goodbye',
 'greeting',
 'help',
 'information_request',
 'name',
 'negative_feedback',
 'noanswer',
 'nutrition_information',
 'positive_feedback',
 'prakriti',
 'sleep_tips',
 'stress_management',
 'thanks']

In [95]:
def getResponse(ints, intents_json):
    tag = ints["intent"]
    list_of_intents = intents_json["intent"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [96]:
getResponse(predict_class("what is your name",model),intents)

1/1 [==============================] - 0s 28ms/step
[4.5445495e-07 1.5017363e-09 2.2133837e-07 8.5058707e-13 5.6947789e-09
 1.2275012e-10 4.0441570e-11 2.7722299e-08 6.2139573e-12 9.5216912e-13
 9.0846436e-12 9.6519370e-09 3.8461040e-10 9.9999893e-01 1.7863953e-10
 3.1761289e-08 4.5803215e-08 2.7517121e-11 2.3078539e-11 2.9247582e-07
 1.3008218e-10 1.1704132e-12]
maximum index = 13 probabiliy =  0.9999989


"I'm AyurBot!"

## Creating the dataset

In [119]:
dataset={
}

In [120]:
for w in words:
    dataset[w]=[]
dataset["tag"]=[]
dataset

{"'m": [],
 "'re": [],
 "'s": [],
 'a': [],
 'about': [],
 'age': [],
 'annoying': [],
 'anyone': [],
 'are': [],
 'away': [],
 'bad': [],
 'be': [],
 'been': [],
 'best': [],
 'by': [],
 'bye': [],
 'ca': [],
 'call': [],
 'can': [],
 'chatbot': [],
 'chatgpt': [],
 'complete': [],
 'contact': [],
 'could': [],
 'created': [],
 'creator': [],
 'customer': [],
 'day': [],
 'despise': [],
 'detail': [],
 'determine': [],
 'diet': [],
 'disappointed': [],
 'do': [],
 'doing': [],
 'eat': [],
 'eating': [],
 'exercise': [],
 'exit': [],
 'fact': [],
 'feeling': [],
 'figure': [],
 'find': [],
 'food': [],
 'for': [],
 'fuck': [],
 'get': [],
 'give': [],
 'go': [],
 'good': [],
 'goodbye': [],
 'great': [],
 'hand': [],
 'hate': [],
 'have': [],
 'healthy': [],
 'hello': [],
 'help': [],
 'helpful': [],
 'hey': [],
 'hi': [],
 'hii': [],
 'hiii': [],
 'how': [],
 'i': [],
 'identify': [],
 'impressed': [],
 'improve': [],
 'in': [],
 'information': [],
 'irrelevant': [],
 'is': [],
 'job'

In [121]:
for train in training_data:
    for w,value in list(zip(words,train[0])):
        dataset[w].append(value)
    dataset["tag"].append(classes[np.argmax(train[1])])
dataset        

{"'m": [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 "'re": [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [122]:
for key in dataset.keys():
    print(key,dataset[key])

'm [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
're [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
's [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
a [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [131]:
index=[]

In [132]:
for docu in documents:
    index.append(" ".join(docu[0]))

In [133]:
index

['Hi',
 'How are you ?',
 'Hey there , how have you been ?',
 'Is anyone there ?',
 'Hello',
 'Good day',
 'Hiii',
 'Hii',
 'Bye',
 'See you there',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That 's helpful",
 'Nice',
 'Good',
 'about you',
 'Who are you ?',
 'What are you ?',
 'Who you are ?',
 'what you can do',
 'What is the Purpose of the project',
 'what is your name',
 'what should I call you',
 'whats your name ?',
 'Could you help me ?',
 'give me a hand please',
 'Can you help ?',
 'What can you do for me ?',
 'I need a support',
 'I need a help',
 'support me please',
 'Complete the work',
 'My name',
 'user name',
 'you are not Good',
 'Nothing',
 'You are Bad',
 'Noting to worry',
 'Chatgpt is Best',
 'Irrelevant',
 'Can you help me find my prakriti',
 'what is my prakriti ?',
 'figure out my prakriti',
 'identify my prakriti',
 'Determine my Prakriti',
 'Determine',
 'Who are the creators',
 'who made these project',
 'Project Creators',
 'project is made by',
 'Project',
 'W

In [134]:
df=pd.DataFrame(dataset,index=index)
df

,'m,'re,'s,a,about,age,annoying,anyone,are,away,...,who,with,work,workout,worry,worthless,yeah,you,your,tag
Hi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,greeting
How are you ?,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,greeting
"Hey there , how have you been ?",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,greeting
Is anyone there ?,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,greeting
Hello,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,greeting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Give me nutrition facts,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nutrition_information
I want to learn about healthy eating,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nutrition_information
exit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,exit
quit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,exit


In [142]:
import os


df.to_csv("../dataset/sentence_dataset.csv")